In [1]:
import shutil
import re 
from glob import glob
import os
from tqdm import tqdm

import librosa
import soundfile as sf
import numpy as np 
import random
import pandas as pd

from functional import *
from overlap import get_trimmed_sample, overlap_using_min

In [2]:
sr = 16000

In [ ]:
paths = {"sample1": "C:/Users/b.smadiarov/Diploma/DualPathRNN/samples/5ed8a1c0f3ea2.flac",
         "sample2": "C:/Users/b.smadiarov/Diploma/DualPathRNN/samples/5f2b0a06ceb18.flac",
         
         'KS2': 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/',
         'root': 'F:/ISSAI_KSC2_unpacked',
         
         'src_csv': 'C:/Users/b.smadiarov/Diploma/VoiceDetectionForKazakhSpeech/data/annotation/Csv', 

         'trimmed': 'F:/ISSAI_KSC2_unpacked/trimmed', 
         "mixed": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/mixed", 
         "samples": "C:/Users/b.smadiarov/Diploma/DualPathRNN/samples"
        }

glob_paths = {'KS2_raw': 'F:/ISSAI_KSC2_unpacked/ISSAI_KSC2/**/**/*.flac',
              "mixed_csv": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/mixed/*.csv",
              "mixed_flac": "C:/Users/b.smadiarov/Diploma/DualPathRNN/data/mixed/*.flac",

              "f_real_flac": 'F:/ISSAI_KSC2_unpacked/trimmed/**/**/*.flac'
             } 

#### Generate samples

- Сначала мы убираем все таки кусочки где начало и конец. И сохраняем все такие файлы в папку targets/
- Затем уже на имеющимся файлами делаем.

ToDO: 
- Короткие записи сливались с короткими записями.

Критерий: нету посторонних звуков 
1. tts - только для text_to_speech
2. Самый лучший crowdsourced,
3. tv_news: эпортаж это обычно речь 
4. parlament: т.к. кричат с требун  
5. Ужасные - radio: т.к. играет вечно музыка, talkshow - тоже какая то дичь, podcast - казахские подкасты это не подкасты а радио.

Замечание: Выбери какие тебе подойдут для лейблов. Но если не подойдет то можно читать датафрейм и искать все такие видео которые имеют три интервала - "не говорит, говорит, не говорит" - поэтому ты и выбрал crowdsourced.

In [4]:
tracklist = get_audios_with_folder_name(['crowdsourced', 'tts', 'tv_news'], glob_paths['KS2_raw'])
print('Dataset total len:', len(tracklist))

  0%|          | 0/645860 [00:00<?, ?it/s]

Dataset total len: 516458


Кроме памяти (мне ее жалко) и так же в аннотационных файлах указана путь до файла. Поэтому что бы придестеречься 
я решил оставить все как есть.

**1. Убираем начало и конец.**

In [5]:
for t in tqdm(tracklist):
    trimmed_audio = get_trimmed_sample(t, sr, paths['src_csv'])
    folder = t.replace("ISSAI_KSC2/", "trimmed/").replace(f"/{get_file_name (t)}.flac", '')
    os.makedirs(folder, exist_ok=True)
    path_to_save = folder + f"/{get_file_name (t)}.flac"
    sf.write(path_to_save, trimmed_audio, samplerate = sr)

  0%|          | 0/516458 [00:00<?, ?it/s]

**2. Объединяем аудиодорожки**

Число класстеров

In [4]:
k = 2
seed = k

_Seed_ зависит от числа кластеров. Это сделанно для того что бы добавить не предусмотренность. Но таким образом есть опасность что модель просто выучит голоса нежели начнет, как будто для каждого числа класстера можно брать разный датасет. 

Перемешивать нужно для того "говорящие" не шли одним за другим.


In [5]:
tracklist = glob(glob_paths['f_real_flac'])

In [6]:
random.seed(seed)
np.random.seed(seed)
random.shuffle(tracklist)

In [7]:
os.makedirs(f"{paths['root']}/mixed_{k}", exist_ok=True)

# Этот метод не нравится т.к. хочется что бы дорожка была как можно длинее.
# То есть аудио с одинаковой длины объединялись с одинаковыми аудио. 
for i in tqdm(range(0, len(tracklist), k)):
    tracks = tracklist[i:i+k]
    tracks = sorted(tracks)
    overlay = overlap_using_min(tracks)
    audio_names = "_".join([get_file_name(t) for t in tracks])
    audio_name = f"{paths['root']}/mixed_{k}/{audio_names}.flac"
    sf.write(f"{audio_name}", overlay, samplerate=16000)
    df = pd.DataFrame({'mixed_audio': [audio_name], **{f'audio_{j+1}': [t.replace('\\', '/')] for j, t in enumerate(tracks)}})
    df.to_csv(f"{audio_name.replace('.flac', '.csv')}", index=False)

  0%|          | 0/258229 [00:00<?, ?it/s]

In [12]:
# Get full dataset DF
annotations = glob(f"F:/ISSAI_KSC2_unpacked/mixed_{k}/*.csv")
print(len(annotations))
all_dfs = []
for csv in tqdm(annotations):
    df = pd.read_csv(csv)
    all_dfs.append(df)

concated_df = pd.concat(all_dfs).reset_index(drop=True)
os.makedirs(f"F:/ISSAI_KSC2_unpacked/DIHARD_DATA_INFO", exist_ok=True)
concated_df.to_csv(f"F:/ISSAI_KSC2_unpacked/DIHARD_DATA_INFO/CONCATED_DFS_k_{k}.csv", index=False)

258229


  0%|          | 0/258229 [00:00<?, ?it/s]

In [13]:
print(f"Mixed num files {len(concated_df)} for k = {k} clusters")

Mixed num files 258229 for k = 2 clusters
